In [1]:
from o2s.models.object_2_sphere import Mesh2Sphere
from o2s.datasets.radar_dataset import RadarDataset
from o2s.datasets.transforms.general import Compose
from o2s.datasets.transforms.radar import Log, Abs, Normalize, Center
from o2s.datasets.transforms.mesh import MeshNormalize

import numpy as np
import matplotlib.pyplot as plt
from torch_harmonics import plotting
from torch_geometric.loader import DataLoader
import xarray
import torch
from torch import nn

In [2]:
transforms = Compose([Abs(), Log()]) 
dataset = RadarDataset('/home/colin/hdd/workspace/object2sphere/datasets/test_dataset.nc', 'test', 'simple', 'full', transforms=transforms, return_mesh=True)

In [3]:
B = 1
dataloader = DataLoader(dataset, batch_size=B, shuffle=True)

In [5]:
model = Mesh2Sphere(5, 30, 64, 10.0, use_mlp=False).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)

/home/colin/anaconda3/envs/torch/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home/colin/anaconda3/envs/torch/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home/colin/anaconda3/envs/torch/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
for batch in dataloader:
    #mesh, resp, vert, face = batch
    
    for iter in range(200):
        pred, _ = model(mesh.cuda())
        target = resp[:,53].cuda()
        weight = nn.functional.softmax(target, dim=1)
        loss = weight * (pred - target) ** 2
        loss = loss.sum().mean()
        #loss = (y_pred - target.view(1,61,21).cuda()).pow(2).mean()
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        if iter % 10 == 0:
            print(f'iteration: {iter} loss: {loss.item()}')
    break

In [ ]:
with torch.no_grad():
    pred, w = model(mesh.cuda())
    plotting.plot_spherical_fn(pred.cpu().squeeze().numpy(), fig=plt.figure(figsize=(2,2)), vmin=target.cpu().min(), vmax=target.cpu().max())

import matplotlib.animation as animation

fig = plt.figure(layout='constrained', figsize=(12, 8), dpi=72)

moviewriter = animation.writers['pillow'](fps=30)
moviewriter.setup(fig, 'pred.gif', dpi=72)

num_frames = 360
for frame in range(num_frames):
    # compute the rotation of the sphere
    lon = -num_frames - (360 / num_frames) * frame

    if frame % 10 == 0:
        print(f"frame={frame}")
    
        plotting.plot_spherical_fn(pred.cpu().squeeze().numpy(), fig=fig, central_longitude=lon,vmin=-0.2, vmax=0.3)

    plt.draw()
    moviewriter.grab_frame()

moviewriter.finish()

In [ ]:
plotting.plot_spherical_fn(target.cpu().squeeze().numpy(), fig=plt.figure(figsize=(2,2)), vmin=target.cpu().min(), vmax=target.cpu().max())

fig = plt.figure(layout='constrained', figsize=(12, 8), dpi=72)

moviewriter = animation.writers['pillow'](fps=30)
moviewriter.setup(fig, 'target.gif', dpi=72)

num_frames = 360
for frame in range(num_frames):
    # compute the rotation of the sphere
    lon = -num_frames - (360 / num_frames) * frame

    if frame % 10 == 0:
        print(f"frame={frame}")
    
        plotting.plot_spherical_fn(target.cpu().squeeze().numpy(), fig=fig, central_longitude=lon,vmin=-0.2, vmax=0.3)

    plt.draw()
    moviewriter.grab_frame()

moviewriter.finish()

In [ ]:
plotting.plot_spherical_fn(np.abs(target.cpu().squeeze().numpy() - pred.cpu().squeeze().numpy()), fig=plt.figure(figsize=(2,2)))

fig = plt.figure(layout='constrained', figsize=(12, 8), dpi=72)

moviewriter = animation.writers['pillow'](fps=30)
moviewriter.setup(fig, 'error.gif', dpi=72)

num_frames = 360
for frame in range(num_frames):
    # compute the rotation of the sphere
    lon = -num_frames - (360 / num_frames) * frame

    if frame % 10 == 0:
        print(f"frame={frame}")
    
        plotting.plot_spherical_fn(np.abs(target.cpu().squeeze().numpy() - pred.cpu().squeeze().numpy()), fig=fig, central_longitude=lon)

    plt.draw()
    moviewriter.grab_frame()

moviewriter.finish()

In [5]:
import trimesh
for batch in dataloader:
    mesh, resp, vert, face = batch

    break
tr_mesh.show()

## Ground Truth
![SegmentLocal](target.gif "segment")

## Prediction
![SegmentLocal](pred.gif "segment")

## Error
![SegmentLocal](error.gif "segment")